# sscan as Flyer

## 1D step scans using sscan record

Use the [sscan record](https://epics.anl.gov/bcda/synApps/sscan/sscanRecord.html) as an [ophyd Flyer](http://nsls-ii.github.io/bluesky/async.html) for [bluesky](http://nsls-ii.github.io/bluesky) data acquisition.  Consider the case of [1D step scans using sscan record](https://epics.anl.gov/bcda/synApps/sscan/sscanRecord.html#HEADING_1-1).

In [3]:
import asyncio
from collections import deque
import numpy as np
import time

%matplotlib notebook
from bluesky.utils import install_qt_kicker
install_qt_kicker()

# common IOC prefix to be used
P = "vm7:"

In [ ]:
from ophyd.scaler import ScalerCH
scaler = ScalerCH(f"{P}scaler1", name="scaler")
scaler.select_channels(None)

In [6]:
from ophyd import EpicsMotor
m1 = EpicsMotor(f"{P}m1", name="m1")

In [4]:
from apstools.synApps_ophyd import userCalcsDevice
calcs = userCalcsDevice(P, name="calcs")

In [ ]:
from apstools.synApps_ophyd import sscanDevice
scans = sscanDevice(P, name="scans")
scans.select_channels()

In [ ]:
from apstools.synApps_ophyd import SaveData
save_data = SaveData(f"{P}saveData_", name="save_data")

In [ ]:
# configure saveData for data collection into MDA files:
        
save_data.file_system.put("/tmp")
save_data.subdirectory.put("saveData")
save_data.base_name.put("sscan1_")
save_data.next_scan_number.put(1)
save_data.comment1.put("testing")
save_data.comment2.put("configured and run from ophyd")

In [ ]:
# configure the sscan record for data collection:

# clear out the weeds
scans.reset()

scan = scans.scan1
scan.number_points.put(6)
scan.positioners.p1.setpoint_pv.put(m1.user_setpoint.pvname)
scan.positioners.p1.readback_pv.put(m1.user_readback.pvname)
scan.positioners.p1.start.put(-1)
scan.positioners.p1.end.put(0)
scan.positioner_delay.put(0.0)
scan.detector_delay.put(0.1)
scan.detectors.d01.input_pv.put(scaler.channels.chan03.s.pvname)
scan.detectors.d02.input_pv.put(scaler.channels.chan02.s.pvname)
scan.triggers.t1.trigger_pv.put(scaler.count.pvname)

# finally, reconfigure
scans.select_channels()

In [ ]:
# TODO: make a plan for bluesky

# probably does not need to be a Flyer
# still, try to make it fit

from ophyd.flyers import FlyerInterface

class SscanFlyer_1D_StepSimple(FlyerInterface):

    def kickoff(self):
        # http://nsls-ii.github.io/ophyd/generated/ophyd.flyers.FlyerInterface.kickoff.html
        pass   # -> StatusBase

    def complete(self):
        # http://nsls-ii.github.io/ophyd/generated/ophyd.flyers.FlyerInterface.complete.html
        pass   # -> StatusBase

    def describe_collect(self):
        # http://nsls-ii.github.io/ophyd/generated/ophyd.flyers.FlyerInterface.describe_collect.html
        pass   # -> Dict[str, Dict]

    def collect(self):
        # http://nsls-ii.github.io/ophyd/generated/ophyd.flyers.FlyerInterface.collect.html
        pass   # -> Generator[Dict, None, None]

    def collect_tables(self):
        pass   # -> Iterable[Any]

## MockFlyer with motor and pseudo-detector

ophyd's *MockFlyer* example : https://github.com/NSLS-II/ophyd/blob/master/ophyd/sim.py#L546

In [ ]:
class MockFlyer:
    """
    Class for mocking a flyscan API implemented with stepper motors.
    """

    def __init__(self, name, detector, motor, start, stop, num, loop=None):
        self.name = name
        self.parent = None
        self._mot = motor
        self._detector = detector
        self._steps = np.linspace(start, stop, num)
        self._data = deque()
        self._completion_status = None
        if loop is None:
            loop = asyncio.get_event_loop()
        self.loop = loop

    def __setstate__(self, val):
        name, detector, motor, steps = val
        self.name = name
        self.parent = None
        self._mot = motor
        self._detector = detector
        self._steps = steps
        self._completion_status = None
        self.loop = asyncio.get_event_loop()

    def __getstate__(self):
        return (self.name, self._detector, self._mot, self._steps)

    def read_configuration(self):
        return OrderedDict()

    def describe_configuration(self):
        return OrderedDict()

    def describe_collect(self):
        dd = dict()
        dd.update(self._mot.describe())
        dd.update(self._detector.describe())
        return {'stream_name': dd}

    def complete(self):
        if self._completion_status is None:
            raise RuntimeError("No collection in progress")
        return self._completion_status

    def kickoff(self):
        if self._completion_status is not None:
            raise RuntimeError("Already kicked off.")
        self._data = deque()

        self._future = self.loop.run_in_executor(None, self._scan)
        st = DeviceStatus(device=self)
        self._completion_status = st
        self._future.add_done_callback(lambda x: st._finished())
        return st

    def collect(self):
        if self._completion_status is None or not self._completion_status.done:
            raise RuntimeError("No reading until done!")
        self._completion_status = None

        yield from self._data

    def _scan(self):
        "This will be run on a separate thread, started in self.kickoff()"
        ttime.sleep(.1)
        for p in self._steps:
            stat = self._mot.set(p)
            while True:
                if stat.done:
                    break
                ttime.sleep(0.01)
            stat = self._detector.trigger()
            while True:
                if stat.done:
                    break
                ttime.sleep(0.01)

            event = dict()
            event['time'] = ttime.time()
            event['data'] = dict()
            event['timestamps'] = dict()
            for r in [self._mot, self._detector]:
                d = r.read()
                for k, v in d.items():
                    event['data'][k] = v['value']
                    event['timestamps'][k] = v['timestamp']
            self._data.append(event)

    def stop(self, *, success=False):
        pass


In [16]:
# make a noisy detector in an EPICS swait record, peak ceneter at 2
noisy_det = calcs.calc2.val

# FIXME: crashes an EPICS 7 IOC
# from apstools.synApps_ophyd import swait_setup_lorentzian
# swait_setup_lorentzian(calcs.calc2, m1, 2)

In [19]:
swait = calcs.calc2
motor = m1
center, width, scale, noise = 2, 1, 1, 0.05

swait.reset()
swait.desc.put("noisy Lorentzian curve")
swait.scan.put("Passive")
swait.channels.A.input_pv.put(motor.user_readback.pvname)
swait.channels.B.value.put(center)
swait.channels.C.value.put(width)
swait.channels.D.value.put(scale)
swait.channels.E.value.put(noise)

swait.calc.put("D*(0.95+E*RNDM)/(1+((A-b)/c)^2)")
# FIXME: swait.scan.put("I/O Intr")     # crashes an EPICS 7 IOC
swait.hints = {"fields": ['val',]}
swait.read_attrs = ['val',]

In [ ]:
mflyer = MockFlyer('mflyer', noisy_det, m1, 1, 5, 20)

In [ ]:
# step-scan the motor and read the noisy detector
if False:     # disable for now (it works)
    t0 = time.time()
    for p in range(10):
        m1.move(p-3)
        print(
            "%8.3f" % (time.time()-t0), 
            "%8.2f" % m1.position, 
            "%8.4f" % noisy_det.get()
             )
    m1.move(0)
    print("Complete in %.3f seconds" % (time.time()-t0))

In [ ]:
from databroker import Broker
db = Broker.named("mongodb_config")

In [ ]:
from bluesky import RunEngine
import bluesky.plans as bp
from bluesky.callbacks.best_effort import BestEffortCallback
from bluesky import SupplementalData

RE = RunEngine({})
RE.subscribe(db.insert)
RE.subscribe(BestEffortCallback())
RE.preprocessors.append(SupplementalData())

In [ ]:
RE = RunEngine({})
RE(bp.scan([noisy_det], m1, -5, 5, 11))

In [ ]:
h = db[-1]
h.table()